In [1]:
import numpy as np
# 역전파
class Variable:
    def __init__(self, data):
        self.data = data
        self.grad = None

In [2]:
class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(y)
        self.input = input   # 입력 변수를 기억 ( 보관 )
        return output
        
def forward(self, x):
        raise NotImplementedError()

def backward(self, gy):
        raise NotImplementedError()

In [3]:
class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y

    def backward(self, gy):
        x = self.input.data
        gx = 2 * x * gy
        return gx

In [4]:
class Exp(Function):
    def forward(self, x):
        y = np.exp(x)
        return y

    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x) * gy
        return gx

In [5]:
# y = (eˣ²)² 순전파 코드
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

In [6]:
# y의 x에 대한 미분 결과
y.grad = np.array(1.0)
b.grad = C.backward(y.grad)
a.grad = B.backward(b.grad)
x.grad = A.backward(a.grad)
print(x.grad)

3.297442541400256


In [7]:
import numpy as np
# 역전파 자동화
class Variable:
    def __init__(self, data):
        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func

In [8]:
class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(y)
        output.set_creator(self)   # 출력 변수에 창조자 성정
        self.input = input   # 입력 변수를 기억 ( 보관 )
        self.output = output   # 출력도 저장
        return output

In [9]:
class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y

    def backward(self, gy):
        x = self.input.data
        gx = 2 * x * gy
        return gx

class Exp(Function):
    def forward(self, x):
        y = np.exp(x)
        return y

    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x) * gy
        return gx

In [10]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

# 계산 그래프르이 노드들을 거꾸로 거슬러 올라감
assert y.creator == C
assert y.creator.input == b
assert y.creator.input.creator == B
assert y.creator.input.creator.input == a
assert y.creator.input.creator.input.creator == A
assert y.creator.input.creator.input.creator.input == x

In [11]:
y.grad = np.array(1.0)   # dy / dy라 무조건 1임

C = y.creator   # 1. 함수를 가져온다.
b = C.input   # 2. 함수의 입력을 가져온다.
b.grad = C.backward(y.grad)   # 3. 함수의 backward 메서드를 호출한다

In [12]:
B = b.creator   # 1. 함수를 가져온다.
a = B.input   #2. 함수의 입력을 가져온다.
a.grad = B.backward(b.grad)   #3. 함수의 backward 메서드를 호출한다.

In [13]:
b.grad = C.backward(y.grad)
a.grad = B.backward(b.grad)
x.grad = A.backward(a.grad)
print(x.grad)

3.297442541400256


In [14]:
class Variable:
    def __init__(self, data):
        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func

    def backward(self):
        # f = self.creator
        # if f is not None:
        #     x = f.input
        #     x.grad = f.backward(self.grad)
        #     x.backward()
        f = self.creator   # 1. 함수를 가져온다.
        if f is not None:
            x = f.input   # 2. 함수의 입력을 가져온다.
            x.grad = f.backward(self.grad)   # 3. 함수의 backward 메서드를 호출한다.
            x.backward()   # 하나 앞 변수의 backward 메서드를 호줄한다. ( 재귀 )

In [15]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

# 역전파
y.grad = np.array(1.0)
y.backward()
print(x.grad)

3.297442541400256
